## Moment Capacity - Unsupported
This example calculates the moment capacity  of a beam subjected to two concentrated forces, and checks the capacity against the applied moments.  Lateral supports are supplied at the ends and at the points of application of the loads.  All segments are checked.

In [1]:
from utils import show, call
from sst import SST
from S16_09 import calcMr

In [2]:
mySST = SST()

![Unsupported Beam Bending](unbraced-2-point-loads.svg)

In [3]:
## input data
L1 = 4.
L2 = 2.5
L3 = 4.
P1F = 120.        # factored loads
P2F = 140.
P1S = 0.4 * P1F   # service loads
P2S = 0.4 * P2F

In [4]:
## calculate reactions and moments
L = L1 + L2 + L3
R3F = (P1F*L1 + P2F*(L1+L2))/L
R0F = P1F + P2F - R3F
M1F = R0F*L1
M2F = R0F*(L1+L2) - P1F*L2
show('R0F,R3F,M1F,M2F')

R0F = 127.6
R3F = 132.4
M1F = 510.5
M2F = 529.5


In [5]:
## trial section and material properties
Dsg = 'W530x74'
trialSec = mySST.section(Dsg)
Fy = 350.
E = 200000.

In [6]:
def sign(x):
    return -1. if x < 0. else (+1. if x > 0. else 0.)
def calcw2(M1,M2):
    k = min(abs(M1),abs(M2))/max(abs(M1),abs(M2))
    if sign(M1) == sign(M2):
        k = -k
    return min(1.75 + 1.05*k + 0.3*k*k,2.5)

In [7]:
# Section 0 - 1
L = L1*1000.
Mf = abs(M1F)
omega2 = calcw2(0.,M1F)
Lu,Mr = call(calcMr,trialSec,Fy=Fy,L=L,omega2=omega2)
MrOK = Mr >= Mf
show('Mf,L,omega2,Lu,Mr,MrOK')

Mf     = 510.5
L      = 4000
omega2 = 1.75
Lu     = 2026
Mr     = 487.9
MrOK   = False


In [8]:
# Section 1 - 2
L = L2*1000.
Mf = max(abs(M1F),abs(M2F))
omega2 = calcw2(M1F,M2F)
Lu,Mr = call(calcMr,trialSec,Fy=Fy,L=L,omega2=omega2)
MrOK = Mr >= Mf
show('Mf,L,omega2,Lu,Mr,MrOK')

Mf     = 529.5
L      = 2500
omega2 = 1.017
Lu     = 2026
Mr     = 530.8
MrOK   = True


In [9]:
# Section 2 - 3
L = L3*1000.
Mf = abs(M2F)
omega2 = calcw2(M2F,0.)
Lu,Mr = call(calcMr,trialSec,Fy=Fy,L=L,omega2=omega2)
MrOK = Mr >= Mf
show('Mf,L,omega2,Lu,Mr,MrOK')

Mf     = 529.5
L      = 4000
omega2 = 1.75
Lu     = 2026
Mr     = 487.9
MrOK   = False


In [10]:
def calcDelta(P,a,l,E=E,I=1.):
    b = l - a
    x = l/2.
    if a < x:
        a,b = b,a
    return P*b*x*(l*l-b*b-x*x)/(6.*E*I*l)

In [11]:
L = (L1+L2+L3)*1000.
delta = calcDelta(P1S*1000.,L1*1000.,L,I=trialSec.Ix) + calcDelta(P2S*1000.,(L1+L2)*1000.,L,I=trialSec.Ix)
ratio = L/delta
show('L,delta,ratio')

L     = 1.05e+04
delta = 28.12
ratio = 373.3
